In [86]:
import pandas as pd
import numpy as np
import requests
import os
import fastf1 as f1

season = 2024
event = 'Monaco'
session_type = 'R'

session = f1.get_session(season, event, session_type)
session.load()
df = session.results.copy()

if session_type in ['R', 'S']:
    df = df[['DriverNumber', 'Abbreviation', 'TeamName', 'Position', 'GridPosition', 'Status']]
    df.rename(columns={
        'DriverNumber': 'car_number',
        'Abbreviation': 'driver',
        'TeamName': 'constructor',
        'Position': 'finish_pos',
        'GridPosition': 'grid_pos',
        'Status': 'status'
    }, inplace=True)
    
elif session_type in ['Q', 'SQ']:
    df = df[['DriverNumber', 'Abbreviation', 'TeamName', 'Q1', 'Q2', 'Q3', 'Position']]
    df.rename(columns={
        'DriverNumber': 'car_number',
        'Abbreviation': 'driver',
        'TeamName': 'constructor',
        'Position': 'finish_pos'
    }, inplace=True)
    
df['season'] = season
df['event'] = event
df['session_type'] = session_type

df

core           INFO 	Loading data for Monaco Grand Prix - Race [v3.6.0]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['16', '81', '55', '4', '63', '1', '44', '22', '23', '10', '14', '3', '77', '18', '2', '24', '31', '11', '27', '20']


,car_number,driver,constructor,finish_pos,grid_pos,status,season,event,session_type
16,16,LEC,Ferrari,1.0,1.0,Finished,2024,Monaco,R
81,81,PIA,McLaren,2.0,2.0,Finished,2024,Monaco,R
55,55,SAI,Ferrari,3.0,3.0,Finished,2024,Monaco,R
4,4,NOR,McLaren,4.0,4.0,Finished,2024,Monaco,R
63,63,RUS,Mercedes,5.0,5.0,Finished,2024,Monaco,R
1,1,VER,Red Bull Racing,6.0,6.0,Finished,2024,Monaco,R
44,44,HAM,Mercedes,7.0,7.0,Finished,2024,Monaco,R
22,22,TSU,RB,8.0,8.0,Lapped,2024,Monaco,R
23,23,ALB,Williams,9.0,9.0,Lapped,2024,Monaco,R
10,10,GAS,Alpine,10.0,10.0,Lapped,2024,Monaco,R


In [92]:
base_url = "https://api.jolpi.ca/ergast/f1"
url = f"{base_url}/{season}/driverstandings.json"
response = requests.get(url)
data = response.json()

standings = data['MRData']['StandingsTable']['StandingsLists'][0]['DriverStandings']

rows = []
for driver in standings:
    rows.append({
        "season": season,
        "driver": driver["Driver"]["code"],
        "car_number": driver["Driver"]["permanentNumber"],
        "constructor": driver["Constructors"][0]["name"],
        "position": driver["position"],
        "points": driver["points"],
        "wins": driver["wins"],
    })

df = pd.DataFrame(rows)
df

,season,driver,car_number,constructor,position,points,wins
0,2024,VER,33,Red Bull,1,437,9
1,2024,NOR,4,McLaren,2,374,4
2,2024,LEC,16,Ferrari,3,356,3
3,2024,PIA,81,McLaren,4,292,2
4,2024,SAI,55,Ferrari,5,290,2
5,2024,RUS,63,Mercedes,6,245,2
6,2024,HAM,44,Mercedes,7,223,2
7,2024,PER,11,Red Bull,8,152,0
8,2024,ALO,14,Aston Martin,9,70,0
9,2024,GAS,10,Alpine F1 Team,10,42,0


In [95]:
url = f"{base_url}/{season}/constructorstandings.json"
response = requests.get(url)
data = response.json()

standings = data['MRData']['StandingsTable']['StandingsLists'][0]['ConstructorStandings']

rows = []
for constructor in standings:
    rows.append({
        "season": season,
        "constructor": constructor["Constructor"]["name"],
        "position": constructor["position"],
        "points": constructor["points"],
        "wins": constructor["wins"],
    })
    
df = pd.DataFrame(rows)
df

,season,constructor,position,points,wins
0,2024,McLaren,1,666,6
1,2024,Ferrari,2,652,5
2,2024,Red Bull,3,589,9
3,2024,Mercedes,4,468,4
4,2024,Aston Martin,5,94,0
5,2024,Alpine F1 Team,6,65,0
6,2024,Haas F1 Team,7,58,0
7,2024,RB F1 Team,8,46,0
8,2024,Williams,9,17,0
9,2024,Sauber,10,4,0


In [98]:
session = f1.get_session(season, event, session_type)
session.load()
df = session.laps.copy()

df = df[['Driver', 'DriverNumber', 'Team', 'LapNumber', 'LapTime', 'Position', 'Sector1Time', 'Sector2Time', 'Sector3Time', 'Stint']]
df.rename(columns={
    'Driver': 'driver',
    'DriverNumber': 'car_number',
    'Team': 'constructor',
    'LapNumber': 'lap_number',
    'LapTime': 'lap_time',
    'Position': 'position',
    'Sector1Time': 'sector1',
    'Sector2Time': 'sector2',
    'Sector3Time': 'sector3',
    'Stint': 'stint'
}, inplace=True)

df['season'] = season
df['event'] = event
df['session_type'] = session_type

df

core           INFO 	Loading data for Monaco Grand Prix - Race [v3.6.0]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info


req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['16', '81', '55', '4', '63', '1', '44', '22', '23', '10', '14', '3', '77', '18', '2', '24', '31', '11', '27', '20']


,driver,car_number,constructor,lap_number,lap_time,position,sector1,sector2,sector3,stint,season,event,session_type
0,LEC,16,Ferrari,1.0,0 days 00:40:56.330000,1.0,NaT,0 days 00:00:46.389000,NaT,1.0,2024,Monaco,R
1,LEC,16,Ferrari,2.0,NaT,1.0,0 days 00:00:26.854000,0 days 00:00:41.743000,NaT,2.0,2024,Monaco,R
2,LEC,16,Ferrari,3.0,0 days 00:01:24.624000,1.0,0 days 00:00:24.007000,0 days 00:00:38.235000,0 days 00:00:22.382000,2.0,2024,Monaco,R
3,LEC,16,Ferrari,4.0,0 days 00:01:22.625000,1.0,0 days 00:00:21.892000,0 days 00:00:38.779000,0 days 00:00:21.954000,2.0,2024,Monaco,R
4,LEC,16,Ferrari,5.0,0 days 00:01:21.795000,1.0,0 days 00:00:21.314000,0 days 00:00:38.513000,0 days 00:00:21.968000,2.0,2024,Monaco,R
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1232,ZHO,24,Kick Sauber,76.0,0 days 00:01:17.173000,16.0,0 days 00:00:20.460000,0 days 00:00:35.927000,0 days 00:00:20.786000,3.0,2024,Monaco,R
1233,OCO,31,Alpine,1.0,NaT,NaN,NaT,NaT,NaT,1.0,2024,Monaco,R
1234,PER,11,Red Bull Racing,1.0,NaT,NaN,NaT,NaT,NaT,1.0,2024,Monaco,R
1235,HUL,27,Haas F1 Team,1.0,NaT,NaN,NaT,NaT,NaT,1.0,2024,Monaco,R
